In [2]:
# !pip install beautifulsoup4
# !pip install html5lib
# !pip install python-whois
!pip install pyspark

     -------------------------------------- 310.8/310.8 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 200.5/200.5 kB 12.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285411 sha256=c839af3206dea3050394490febd9262acb14f3282614069b70eff550980a068a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2b\9a\39\d8019ffbfb76a39433455e3d5799e94d3e3cae8f41229f6bf8
Successfully built pyspark


In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

import whois

In [3]:
# gets html of webpage
url = 'https://www.secureworks.com/blog/opsec-mistakes-reveal-cobalt-mirage-threat-actors'
result = requests.get(url)
soup = BeautifulSoup(result.text, "html.parser")

#extracts IP addresses, URLs and hashes from table
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
display(df)

,Indicator,Type,Context
0,gupdate.us,Domain name,TunnelFish C2 server used by COBALT MIRAGE
1,msupdate.top,Domain name,TunnelFish C2 server used by COBALT MIRAGE
2,193.142.59.174,IP address,Hosting TunnelFish domains used by COBALT MIRAGE
3,172.245.26.118,IP address,Staging and distributing COBALT MIRAGE malware
4,mssync.one,Domain name,Suspected C2 server linked to COBALT MIRAGE
5,upmirror.top,Domain name,Suspected C2 server linked to COBALT MIRAGE
6,104.168.117.149,IP address,Hosting COBALT MIRAGE domains
7,69314c1969f28bfab34683769286326e25d9a0f07 c4b...,SHA256 hash,TunnelFish malware used by COBALT MIRAGE
8,f38f3a1cda90229434e8ab8c59342838106b9778,SHA1 hash,TunnelFish malware used by COBALT MIRAGE
9,00e4c488558492b80fd27d51b159a099,MD5 hash,TunnelFish malware used by COBALT MIRAGE


In [5]:
#removes all whitespaces and display results in a list
extracted = [x.replace(" ", "") for x in df['Indicator']]
print(extracted)

['gupdate.us', 'msupdate.top', '193.142.59.174', '172.245.26.118', 'mssync.one', 'upmirror.top', '104.168.117.149', '69314c1969f28bfab34683769286326e25d9a0f07c4bad3443d08efe4f43e0a8', 'f38f3a1cda90229434e8ab8c59342838106b9778', '00e4c488558492b80fd27d51b159a099']


In [68]:
#gets domain and ip addresses 
df_domains = df.loc[df['Type'].isin(['Domain name', 'IP address'])]
domains = df_domains['Indicator']
# print(domains)

#empty df to store results
results_df = pd.DataFrame()

for domain in domains:
    #extracting whois information for each domain
    whois_info = whois.whois(domain)
    whois_info_df = pd.DataFrame(whois_info.items())
    
    #transpose df for csv format
    whois_info_df = whois_info_df.T
    #set header and reset index
    whois_info_df = whois_info_df.rename(columns=whois_info_df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    
    #add domain extracted as new column
    whois_info_df.insert(0, 'Domain/IP', domain)
    
    #append results
    results_df = pd.concat([results_df, whois_info_df], ignore_index=True)

#export to csv, fill NA values
display(results_df)
results_df.to_csv("taskA.csv", encoding='utf-8', index=False, na_rep='NA')

0         gupdate.us
1       msupdate.top
2     193.142.59.174
3     172.245.26.118
4         mssync.one
5       upmirror.top
6    104.168.117.149
Name: Indicator, dtype: object


,Domain/IP,domain_name,domain__id,whois_server,registrar,registrar_id,registrar_url,registrar_email,registrar_phone,status,...,updated_date,referral_url,emails,dnssec,name,org,address,city,state,country
0,gupdate.us,gupdate.us,DD745E60FBAD445BBA4F822873796C6A3-GDREG,whois.namecheap.com,NAMECHEAP INC,1068,http://www.namecheap.com,abuse@namecheap.com,"[+1.6613102107, +1.9854014545]",[clientTransferProhibited https://icann.org/ep...,...,"[2023-06-06 09:22:11, 0001-01-01 00:00:00]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,msupdate.top,None,NaN,None,None,NaN,NaN,NaN,NaN,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,193.142.59.174,None,NaN,None,None,NaN,NaN,NaN,NaN,ASSIGNED PA,...,None,NaN,"[abuse@ripe.net, hostmaster@ripe.net, 'abuse@h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,172.245.26.118,"[COLOCROSSING.COM, colocrossing.com]",NaN,whois.names4ever.com,"Deluxe Small Business Sales, Inc. d/b/a Aplus.net",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-06-22 15:16:46,NaN,"[dns@cs.aplus.net, dns@aplus.net, corp-domains...","[unsigned, Unsigned]",Domain Admin,"Deluxe Small Business Sales, Inc. Aplus.net",801 Marquette Ave S 801 Marquette Ave S,Minneapolois,MN,US
4,mssync.one,mssync.one,NaN,https://porkbun.com/whois,Porkbun,NaN,NaN,NaN,NaN,[pendingDelete https://icann.org/epp#pendingDe...,...,2023-08-09 12:04:40,NaN,abuse@porkbun.com,unsigned,REDACTED FOR PRIVACY,"Private by Design, LLC",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,NC,US
5,upmirror.top,upmirror.top,NaN,whois.namesilo.com,"NameSilo,LLC",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-08-02 17:33:53,NaN,abuse@namesilo.com,unsigned,REDACTED FOR PRIVACY,See PrivacyGuardian.org,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,AZ,US
6,104.168.117.149,"[COLOCROSSING.COM, colocrossing.com]",NaN,whois.names4ever.com,"Deluxe Small Business Sales, Inc. d/b/a Aplus.net",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-06-22 15:16:46,None,"[dns@cs.aplus.net, dns@aplus.net, corp-domains...","[unsigned, Unsigned]",Domain Admin,"Deluxe Small Business Sales, Inc. Aplus.net",801 Marquette Ave S 801 Marquette Ave S,Minneapolois,MN,US


In [59]:
# import session_info
# session_info.show() python-whois-0.8.0

In [11]:
x = 0
all_ips = []
with open("http.log") as infile:
    for line in infile:
#         print(line)
        x+=1
        fields = line.split('\t')
        source_ip = fields[2]
        if source_ip not in all_ips:
            all_ips.append(source_ip)
print(x)
print(len(all_ips))

2048442
71


By inspection, each line of the log file contains fields such as:
Timestamp
Unique Request ID
Source IP
Source Port Number
Destination IP
Destination Port Number
HTTP Request Method 
Domain Name
Requested Path
User-Agent info
HTTP Response Code (and status)

Assumptions:
The enterprise web server has a decently robust cybersecurity defence set up, hence only HTTP error codes will be considered. attackers have not found a way to send malicious request (200), the server does not redirect upon errors(302)

There are some conditions that are more obvious,  
If the User-Agent contains keywords of common scanning and recon tools such as "Nmap" or "DirBuster" etc, or "Nikto" "Gobuster", immediately flag as suspicious.

If there are sensitive path access attempts, such as  "/etc/passwd" or "/admin"

Next, if there are a large number of bad request attempts (error code >400), it may indicate that there are some brute force efforts aimed at discovering vulnerabilities or sensitive information on the web server. 

500-internal server error

We can determine by setting a threshold value that flags IPs as suspicious when the number of attempts is suspiciously high.


IP Frequency Analysis:

Calculate the frequency of requests from each IP address over a specific time window (e.g., one hour).
Flag IP addresses with an abnormally high request frequency compared to the average. High-frequency requests could indicate automated scanning.

In [21]:
# Initialize data structures
results = []  # List to store suspicious ips
potentially_suspicious_ips = {} #Dictionary with ip as key and number of times flagged as value

common_scanning_tools = ["Nmap","DirBuster","Nikto","GoBuster"]
sensitive_paths = ["/etc/passwd", "/admin","/login/admin","/admin.php"]
http_error_codes = ["204","400","401","403","404","429","500","503"]
suspicious_threshold = 10

#Reading each entry of log file
with open("http.log") as log_file:
    for line in log_file:
        fields = line.split('\t')
        source_ip = fields[2]
#         http_method = fields[7]
        requested_path = fields[9]
        user_agent = fields[11]
        response_code = fields[14]
        
        # Check for reconnaissance tools in user-agent
        if source_ip not in results:
            for tool in common_scanning_tools:
                if tool in user_agent:
                    results.append(source_ip)
                else:
                    pass
        else:
            continue
            
        
        # Check for sensitive path access
        for path in sensitive_paths:
            if path in requested_path:
                if source_ip not in potentially_suspicious_ips:
                    potentially_suspicious_ips[source_ip] = 1
                else:
                    potentially_suspicious_ips[source_ip] += 1
                break
                    
        # Check for HTTP codes of interest
        if response_code in http_error_codes:
            if source_ip not in potentially_suspicious_ips:
                potentially_suspicious_ips[source_ip] = 1
            else:
                potentially_suspicious_ips[source_ip] += 1
                
        # Filter highly suspicious ips based on threshold
        for source_ip,suspicious_count in potentially_suspicious_ips.items():
            if suspicious_count > suspicious_threshold and source_ip not in results:
                results.append(source_ip)

print("Suspicious IPs:", results)

Suspicious IPs: ['192.168.202.79', '192.168.202.96', '192.168.202.102', '192.168.202.90', '192.168.203.45', '192.168.202.110', '192.168.202.76', '192.168.204.45', '192.168.202.103', '192.168.203.61', '192.168.202.94', '192.168.202.112', '192.168.202.108', '192.168.202.118', '192.168.203.63', '192.168.202.100', '192.168.202.101', '192.168.202.122', '192.168.202.125', '2001:dbb:c18:202:20c:29ff:fe93:571e', '192.168.203.64', '192.168.202.4', '192.168.202.65', '192.168.204.70', '192.168.202.138', '192.168.202.140', '2001:dbb:c18:202:20c:29ff:fe41:4be7', '192.168.202.144', '192.168.202.136', '192.168.202.68', '192.168.202.141', '192.168.202.143', '192.168.202.150']


In [4]:
# Initialize data structures
ip_request_count = {}  # Dictionary to track request counts per IP
ip_non_existing_count = {}  # Dictionary to track non-existing path requests per IP
suspicious_ips = []  # List to store suspicious IPs

#     for line in infile:
# Read and parse log file
with open("http.log") as log_file:
    for line in log_file:
        fields = line.split('\t')
        source_ip = fields[2]
        http_method = fields[7]
        requested_path = fields[10]
        user_agent = fields[12]
        response_code = fields[14]

        # Check for reconnaissance tools in user-agent
        if "Nmap" in user_agent or "DirBuster" in user_agent or "Nikto" in user_agent:
            suspicious_ips.append(source_ip)

        # Track IP frequencies
        ip_request_count[source_ip] = ip_request_count.get(source_ip, 0) + 1

        # Check for sensitive path access
        sensitive_paths = ["/etc/passwd", "/admin"]  # Example sensitive paths
        if any(path in requested_path for path in sensitive_paths):
            suspicious_ips.append(source_ip)

        # Count non-existing path requests
        if response_code == "404":
            ip_non_existing_count[source_ip] = ip_non_existing_count.get(source_ip, 0) + 1

# Check threshold criteria and mark suspicious IPs
request_threshold = 100  # Example request threshold
non_existing_threshold = 10  # Example non-existing path threshold
for ip in ip_request_count:
    if ip_request_count[ip] > request_threshold or ip_non_existing_count.get(ip, 0) > non_existing_threshold:
        suspicious_ips.append(ip)

# Remove duplicates from the suspicious IP list
suspicious_ips = list(set(suspicious_ips))

# Output the list of suspicious IPs
print("Suspicious IPs:", len(suspicious_ips))
print("Suspicious IPs:", suspicious_ips)

Suspicious IPs: 43
Suspicious IPs: ['192.168.202.141', '192.168.202.109', '192.168.203.62', '192.168.203.64', '192.168.202.95', '192.168.202.4', '192.168.202.101', '192.168.202.100', '192.168.204.70', '192.168.202.76', '192.168.202.110', '192.168.202.80', '192.168.202.140', '192.168.202.90', '192.168.202.122', '192.168.202.94', '192.168.202.118', '2001:dbb:c18:202:20c:29ff:fe41:4be7', '192.168.203.45', '192.168.28.100', '192.168.202.64', '192.168.202.143', '2001:dbb:c18:202:20c:29ff:fe93:571e', '192.168.203.61', '192.168.202.87', '192.168.202.96', '192.168.202.63', '192.168.202.102', '192.168.202.108', '192.168.202.68', '192.168.202.62', '192.168.204.45', '192.168.202.103', '192.168.202.97', '192.168.203.63', '192.168.202.144', '192.168.202.125', '192.168.202.112', '192.168.202.136', '192.168.202.65', '192.168.202.150', '192.168.202.138', '192.168.202.79']
